Системы аналитических вычислений.

Студент: Пермяков Никита Александрович, М8О-208Б-19

Лабораторная работа № 2

### Задание 1. Вариант А. дихотомия и метод Ньютона

Поиск решения осуществляется путём построения последовательных приближений и основан на принципах простой итерации. Метод обладает квадратичной сходимостью. 

In [7]:
# Дано
f(x) = x^3 - 9*x^2 + 23*x - 15
a = (0, 2, 0.5)
b = (0, 2, 0.5)
step = (1, 2, 1)

xmin = -1
xmax = 6
ymin = -4
ymax = 5

eps = 1e-5

In [8]:
# Решим уравнение
solve([f(x) == 0], x)

[x == 1, x == 5, x == 3]

In [22]:
# Метод дихотомии - ищем точку пересечения с Ox
def dichotomy(fun, a, b, eps=1e-5):
    if fun(x=a) * fun(x=b) > 0:
        return None
    m = (a + b) / 2.
    if abs(fun(x=m).n()) < eps:
        return m
    if fun(x=m) * fun(x=a) < 0:
        return dichotomy(fun, a, m)
    else:
        return dichotomy(fun, m, b)

In [23]:
# Метод Ньютона - ищем точку пересечения с Ox
def newton(fun, a, b, eps=1e-5):
    x1 = a
    x2 = (a + b) / 2
    df = fun.derivative()
    while abs(x2 - x1) >= eps:
        x1 = x2
        x2 = x1 - fun(x=x1) / df(x=x1)
    if abs(fun(x=x1)) <= 1e-3:
        return x1
    return None

In [24]:
def setup_graph(func):
    return point((float(func), 0), color='green', aspect_ratio=1, size=30)

@interact
def func_interact(a=a, b=b, step=step):    
    # f, eps - global
    
    # дихотомия
    di = [dichotomy(f, a, b, eps)]
    # Добавляем по шагу
    di.append(dichotomy(f, a + step, b + step, eps))
    di.append(dichotomy(f, a + 2 * step, b + 2 * step, eps))
    
    # Ньютон
    newt = [newton(f, a, b, eps)]
    # Добавляем по шагу
    newt.append(newton(f, a + step, b + step, eps))
    newt.append(newton(f, a + 2 * step, b + 2 * step, eps))
    
    
    # устанавливаем параметры 
    graph = [plot(f, (x, xmin, xmax), ymin=ymin, ymax=ymax, aspect_ratio=1, figsize=[3,2], color='black')] * 6
    
    for i in range(len(graph)):
        if i <= 2 and di[i] is not None:
            graph[i] += setup_graph(di[i])
        if i > 2 and newt[i-3] is not None:
            graph[i] += setup_graph(newt[i-3])

    
    str_ = "Корни, полученные методом дихотомии: "
    for d in di:
        if d is not None:
            str_ += str(round(float(d))) + " "
            
    str_ += "\nКорни, полученные методом Ньютона: "
    for n in newt:
        if n is not None:
            str_ += str(round(float(n))) + " "
            
            
    # рисуем
    show("метод дихотомии")        
    graphics_array([[graph[0], graph[1], graph[2]]]).show(frame=True)
    show("метод Ньютона")
    graphics_array([[graph[3], graph[4], graph[5]]]).show(frame=True)            
    show(str_)

Interactive function <function func_interact at 0x7fdaab89e4c0> with 3 widgets
  a: FloatSlider(value=1.0, min…

 результат, полученный методом дихотомии и Ньютона совпадает с результатом solve для каждого корня

### Задание 2. Вариант 2. полиномы и расширенный алгоритм Евклида

In [156]:
f(x) = 5*x^4 - 15*x^3 + 15*x^2 - 5*x
g(x) = -22*x^5 + 33*x^4 - 11*x^3 - 99*x + 99

show(LatexExpr('Дано:'))
show(LatexExpr(r'f(x) = ' + latex(f(x))))
show(LatexExpr(r'g(x) = ' + latex(g(x))))

Дано:

f(x) = 5 \, x^{4} - 15 \, x^{3} + 15 \, x^{2} - 5 \, x

g(x) = -22 \, x^{5} + 33 \, x^{4} - 11 \, x^{3} - 99 \, x + 99

In [162]:
# Создаем кольцо многочленов с переменными x, y
R, (x, y) = PolynomialRing(QQ,'x, y').objgens()
f = 5*x^4 - 15*x^3 + 15*x^2 - 5*x
g = -22*x^5 + 33*x^4 - 11*x^3 - 99*x + 99

In [163]:
# НОД при помощи стандартной функции gcd
show("НОД = ", gcd(f, g))

'НОД = ' x - 1

In [178]:
# Вычисление коэффициентов тождества Безу и gcd(a,b)
def bezout(a, b):
    x = 1
    xx = 0
    y = 0
    yy = 1
    while b:
        entire, remainder = a // b, a % b
        a = b
        b = remainder
        x, xx = xx, x - xx * entire
        y, yy = yy, y - yy * entire
    return (x, y, a)

In [179]:
# результат три элемента: u, v, gcd(a,b)
bez = bezout(f, g)

In [183]:
# левая часть тождества
print("НОД = ", f * bez[0] + g * bez[1])

НОД = -6336/49*x + 6336/49


In [184]:
# правая часть тождества
print("НОД = ", bez[2])

НОД = -6336/49*x + 6336/49


In [185]:
#проверка тождества Безу f*u + v*g = gcd(f, g)
if (f*bez[0]+bez[1]*g) - bez[2] == 0:
    print("Получено верное тождество")
else:
    print("Получено неверное тождество")

Получено верное тождество
